<h1>Crypto Vs. Stocks</h1>
<h3>Group 16:</h3>
<ul>
  <li>Max Brehmer</li>
  <li>Azeem Momin</li>
  <li>Jonathon Shashoua</li>
</ul>

In [1]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
#\/\/\/use this when you put the zip file in (only needs to be ran once)
#!unzip Data.zip

In [2]:
btc = pd.read_csv('/content/Data/BTC.csv', parse_dates=['Date'])
del btc['Adj Close']
btc['Type'] = ['BTC'] * len(btc.index)

nasdaq = pd.read_csv('/content/Data/Nasdaq.csv', parse_dates=['Date'])
del nasdaq['Adj Close']
nasdaq['Type'] = ['Nasdaq'] * len(nasdaq.index)



In [3]:
dji = pd.read_csv('/content/Data/Dow Jones.csv', parse_dates=['Date'])
dji['Type'] = ['Dow Jones'] * len(dji.index)

sp = pd.read_csv('/content/Data/S&P.csv', parse_dates=['Date'])
sp['Type'] = ['S&P'] * len(sp.index)

eth = pd.read_csv('/content/Data/ETH.csv', parse_dates=['Date'])
del eth['Adj Close']
eth['Type'] = ['ETH'] * len(eth.index)

bnd30 = pd.read_csv('/content/Data/Bonds 30yr.csv', parse_dates=['Date'])
del bnd30['Adj Close']
bnd30['Type'] = ['30yr Bonds'] * len(bnd30.index)

bnd10 = pd.read_csv('/content/Data/Bonds 10yr.csv', parse_dates=['Date'])
bnd10['Type'] = ['10yr Bonds'] * len(bnd10.index)

re = pd.read_csv('/content/Data/RealEstate.csv', parse_dates=['Date'])
re['Type'] = ['Real Estate'] * len(re.index)

In [4]:
dates = eth['Date']

btc = btc[btc.Date.isin(dates)]

nasdaq = nasdaq[nasdaq.Date.isin(dates)]

dji = dji[dji.Date.isin(dates)]

sp = sp[sp.Date.isin(dates)]

eth = eth[eth.Date.isin(dates)]

bnd30 = bnd30[bnd30.Date.isin(dates)]

bnd10 = bnd10[bnd10.Date.isin(dates)]

re = re[re.Date.isin(dates)]

In [5]:
frames = [btc, nasdaq, dji, sp, eth, re]
#frames = [btc, nasdaq, dji, sp, eth]

market = pd.concat(frames)

market['Date'] = pd.to_datetime(market['Date'], format="%Y-%m-%d")

market['Yield'] = (market['Close'] - market['Open'])/market['Open']

In [52]:

def render(start_end, graph, width_height):
  
  mask = (market['Date'] >= start_end[0]) & (market['Date'] <= start_end[1])

  year = market.loc[mask]

  w = width_height[0]
  h = width_height[1]

  nearest = alt.selection_single(nearest=True, on='mouseover',
                    encodings=['x','y'], empty='none')

  #Selector for Type
  message = [''] * 6
  Type = pd.DataFrame({'Type' : market.Type.unique(),
                       'Message' : message})
  selection = alt.selection_multi(fields=['Type'])
  color = alt.condition(selection, alt.Color('Type:N'), alt.value('lightgray'))
  type_selector = alt.Chart(Type).mark_rect().encode(
      alt.X('Type', title='', axis=alt.Axis(tickSize=0, labelAngle= 0)),
      color=color,
      tooltip= [alt.Tooltip('Message', title='shift-click to select multiple')]
  ).properties(title='Investment Type', width=300
  ).add_selection(selection)

  #Date Selection
  if graph == 'Prices':
    data = 'Close:Q'
  if graph == 'Yields':
    data = 'Yield:Q'
  if graph == 'Volumes':
    data = 'Volume:Q'
  brush = alt.selection(type='interval', encodings=['x'])

  interval = alt.Chart(year).mark_line(interpolate="basis").encode(
      x = alt.X('Date:T', title='', axis=alt.Axis(tickCount=12)),
      y = alt.Y(data, axis=None),
      color='Type'
  ).properties(
      width=1300, height=50, title='Time Interval'
  ).add_selection(brush).transform_filter(selection)

  interval2 = alt.Chart(year).mark_line(interpolate="basis").encode(
    x = alt.X('Date:T'),
    y = alt.Y(data, axis=None),
    color='Type'
  ).properties(
    width=1300, height=50
  ).transform_filter(selection)

  point = interval2.mark_point().encode(
    tooltip= ['Date:T'],
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
  ).add_selection(nearest)


#Volume Graph
  Volume = alt.Chart(year).mark_line().encode(
      x=alt.X('Date:T', scale=alt.Scale(domain=brush), axis=alt.Axis(labelAngle=-90, tickCount=15)),
      y=alt.Y('Volume:Q', axis=alt.Axis(labelLimit=30, format=',.0f', title='Volume (Billions)')),
      color=alt.Color('Type', legend=None)
  ).properties(
      width=w, height=h, title='Volumes'
  ).transform_filter(selection)

  point1 = Volume.mark_point().encode(
    tooltip= ['Type:O', 'Date:T', alt.Tooltip('Volume:Q', format=',.0f')],
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
  ).add_selection(nearest)

#Price Graph
  Price = alt.Chart(year).mark_line().encode(
      x=alt.X('Date:T', scale=alt.Scale(domain=brush), axis=alt.Axis(labelAngle=-90, tickCount=15)),
      y=alt.Y('Close:Q', axis=alt.Axis(format='$,.0f', title='Price (USD)')),
      color = alt.Color('Type')
  ).properties(
      width=w, height=h, title='Prices' 
  ).transform_filter(selection)

  point2 = Price.mark_point().encode(
    tooltip= ['Type:O', 'Date:T', alt.Tooltip('Close:Q', format='$,.0f', title='Price')],
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
  ).add_selection(nearest)


#Yield Graph
  Yield = alt.Chart(year).mark_line().encode(
      x=alt.X('Date:T', scale=alt.Scale(domain=brush), axis=alt.Axis(labelAngle=-90, tickCount=15)),
      y=alt.Y('Yield:Q', axis=alt.Axis(format='%', title='Percent Yield')),
      color = 'Type'
  ).properties(
  width=w, height=h, title='Yields'
  ).transform_filter(selection)

  point3 = Yield.mark_point().encode(
    tooltip= ['Type:O', 'Date:T', alt.Tooltip('Yield:Q', format='.2%')],
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
  ).add_selection(nearest)


  #display
  if graph == 'Prices':
    return (type_selector & (point + interval)) & ((Price + point2) | (Yield + point3) | (Volume + point1))
  if graph == 'Yields':
    return (type_selector & (point + interval)) & ((Yield + point3) | (Price + point2) | (Volume + point1))
  if graph == 'Volumes':
    return (type_selector & (point + interval)) & ((Volume + point1) | (Price + point2) | (Yield + point3))


In [54]:
year = [('2021-1-1', '2021-11-01'), ('2020-1-1', '2020-12-31'), 
      ('2019-1-1', '2019-12-31'), ('2018-1-1', '2018-12-31'),
      ('2017-1-1', '2017-12-31'), ('2016-1-1', '2016-12-31'),
      ('2015-08-07', '2015-12-31')]
def size(s):
  if s == 'Big':
    return (1300, 400)
  else:
    return (382, 200)

@interact
def render_graphs(Year=[2021, 2020, 2019, 2018, 2017, 2016, 2015], Graph=['Prices', 'Yields', 'Volumes'], Size = ['Big', 'Small']
):
    return render(year[abs(Year - 2021)], Graph, size(Size))



interactive(children=(Dropdown(description='Year', options=(2021, 2020, 2019, 2018, 2017, 2016, 2015), value=2…